# Importing the necessary Libraries

In [1]:
import os, json, joblib, numpy as np, pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from keras.utils import Sequence, to_categorical, pad_sequences
from keras.models import Model, load_model
from keras.layers import (
    Input, Conv1D, BatchNormalization, Activation, add, MaxPooling1D, Dropout,
    Bidirectional, LSTM, GlobalAveragePooling1D, Dense, Multiply, Reshape,
    Lambda, Concatenate, GRU, GaussianNoise
)
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import backend as KerasTensor
import tensorflow as tf
import polars as pl
from sklearn.model_selection import StratifiedGroupKFold
from scipy.spatial.transform import Rotation as R

2025-08-27 07:34:17.699907: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-27 07:34:17.930434: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-27 07:34:20.010775: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Fix Seed

In [2]:
import random
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
seed_everything(seed=42)

# Configuration

In [4]:
import sys
from pathlib import Path

# ~/MyKaggle をモジュールパスに追加
PROJECT_ROOT = Path.home() / "kaggle-cmi-detect" # /home/jiwon/MyKaggle
sys.path.append(str(PROJECT_ROOT))

from config import RAW_DIR, PROCESSED_DIR # /home/jiwon/MyKaggle/competitions/cmi_detect_behavior/config.pyを探せるようになる



import pandas as pd
train = pd.read_csv(RAW_DIR / "train.csv")

if PROCESSED_DIR:
    print("Pretrained model dir:", PROCESSED_DIR)


Pretrained model dir: /home/jiwon/kaggle-cmi-detect/data/processed


In [ ]:
import sys
from pathlib import Path

# ~/MyKaggle をモジュールパスに追加
PROJECT_ROOT = Path.home() / "MyKaggle" # /home/jiwon/MyKaggle
sys.path.append(str(PROJECT_ROOT))

from competitions.cmi_detect_behavior.config import RAW_DIR, PRETRAINED_DIR

Train = False
RAW_DIR = Path(RAW_DIR)
PROCESSED_DIR = Path(PROCESSED_DIR)
